In [14]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline

In [15]:
ailsa = pd.read_csv('Data/ailsa/Ailsadata2.csv', index_col=0)

/usr/local/anaconda3/lib/python3.9/site-packages/IPython/core/interactiveshell.py:3444: DtypeWarning: Columns (8) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


In [16]:
ailsa.head()

,TransactionID,isFraud,TransactionAmt,ProductCD,card3,card4,card6,addr2,dist1,P_emaildomain,...,M1,M2,M3,M4,M5,M6,M7,M8,M9,hour
1,2987000,0,68.5,W,150,discover,credit,87,19.000000,google,...,1,1,1,M2,0,1,0,0,1,0
2,2987001,0,29.0,W,150,mastercard,credit,87,118.502948,google,...,1,1,1,M0,1,1,0,0,1,0
3,2987002,0,59.0,W,150,visa,debit,87,287.000000,Microsoft,...,1,1,1,M0,0,0,0,0,0,0
4,2987003,0,50.0,W,150,mastercard,debit,87,118.502948,yahoo,...,1,1,1,M0,1,0,0,0,1,0
5,2987004,0,50.0,H,150,mastercard,credit,87,118.502948,google,...,1,1,1,M0,0,0,0,0,1,0


In [17]:
ailsa.nunique()

TransactionID     590538
isFraud                2
TransactionAmt     20901
ProductCD              5
card3                  3
card4                  4
card6                  4
addr2                  3
dist1               2652
P_emaildomain         26
C1                  1657
C2                  1216
C3                    27
C4                  1260
C5                   319
C6                  1328
C7                  1103
C8                  1253
C9                   205
C10                 1231
C11                 1476
C12                 1199
C13                 1597
C14                 1108
D1                   642
D2                   642
D3                   650
D4                   809
D5                   689
D10                  819
D11                  677
D15                  860
M1                     2
M2                     2
M3                     2
M4                     3
M5                     2
M6                     2
M7                     2
M8                     2


In [18]:
for col in ailsa.select_dtypes(exclude=np.number).columns.to_list():
    print(ailsa[col].unique())

['W' 'H' 'C' 'S' 'R']
['150' 'other' '185']
['discover' 'mastercard' 'visa' 'american express']
['credit' 'debit' 'debit or credit' 'charge card']
['87' 'other' 87]
['google' 'Microsoft' 'yahoo' 'mail.com' 'anonymous.com' 'verizon.net'
 'aol.com' 'me.com' 'comcast.net' 'optonline.net' 'cox.net' 'charter.net'
 'rocketmail.com' 'other' 'icloud.com' 'att.net' 'juno.com' 'ymail.com'
 'sbcglobal.net' 'bellsouth.net' 'msn.com' 'earthlink.net'
 'roadrunner.com' 'windstream.net' 'mac.com' 'aim.com']
['M2' 'M0' 'M1']


In [19]:
temp = ailsa.select_dtypes(exclude=np.number).astype(str).copy()

cols = ailsa.select_dtypes(exclude=np.number).columns.to_list()

ailsa[cols] = temp

In [20]:
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder
from sklearn.base import BaseEstimator, TransformerMixin


class emailTransformer(BaseEstimator, TransformerMixin):
    def __init__(self):
        return

    def fit(self, X, y=None):
        return self

    def transform(self, X):
        X_ = X.copy()

        temp = X_['P_emaildomain'].copy()

        valid_emails = ['gmail.com', 'hotmail.com', 'anonymous.com', 'yahoo.com']

        temp[~(temp.isin(valid_emails))] = 'other'

        X_['P_emaildomain'] = temp

        return X_

class cat_to_string(BaseEstimator, TransformerMixin):
    def __init__(self):
        return

    def fit(self, X, y=None):
        return self

    def transform(self, X):

        print(type(X))

        X_ = X.copy()

        temp = X_.select_dtypes(exclude=np.number).astype(str)

        cols = X_.select_dtypes(exclude=np.number).columns.to_list()

        X_[cols] = temp

        return X_

In [40]:
num_pipeline = Pipeline([
    ('imputer', SimpleImputer(fill_value=-999999))
])

In [41]:
cat_pipeline = Pipeline([
    ('imp', SimpleImputer(strategy='most_frequent')),
    ('onehot', OneHotEncoder()),
    #('catstring', cat_to_string())
])

In [42]:
from sklearn.compose import ColumnTransformer

num_attribs = ailsa.select_dtypes(include=np.number).columns.to_list()
cat_attribs = ailsa.select_dtypes(exclude=np.number).columns.to_list()

num_attribs.remove('isFraud')

first_pipeline = Pipeline([
    ('email', emailTransformer()),
])

final_pipeline = ColumnTransformer([
    ('num', num_pipeline, num_attribs),
    ('cat', cat_pipeline, cat_attribs)
])

In [43]:
y = ailsa['isFraud']

X = ailsa.drop('isFraud', axis=1)

X_ = first_pipeline.transform(X)

X_ = final_pipeline.fit_transform(X_)

In [44]:
from sklearn.ensemble import RandomForestClassifier

forest = RandomForestClassifier(n_estimators=100, max_depth=None)

In [45]:
#forest.fit(X_, y)

In [46]:
from joblib import dump, load

#dump(forest, 'Data/ailsa/rfmodel.joblib')

In [47]:
sub = ailsa[:int(len(ailsa)*0.1)]
sub_train = sub[:int(len(sub)*0.8)]
sub_test = sub[int(len(sub)*0.8):]

sub_y_train = sub_train['isFraud']

sub_x_train = sub_train.drop('isFraud', axis=1)

sub_x_train = first_pipeline.transform(sub_x_train)

sub_x_train = final_pipeline.fit_transform(sub_x_train)

sub_y_test = sub_test['isFraud']

sub_x_test = sub_test.drop('isFraud', axis=1)

sub_x_test = first_pipeline.transform(sub_x_test)

sub_x_test = final_pipeline.transform(sub_x_test)

In [53]:
from sklearn.metrics import confusion_matrix, f1_score, roc_auc_score

for i in range(4, 20, 1):
    
    clf = RandomForestClassifier(n_estimators=i,criterion='gini', max_depth=20, max_features=8, bootstrap=False)
    
    clf.fit(sub_x_train, sub_y_train)
    
    p = clf.predict(sub_x_test)
    
    print("i: ", i)
    print()
    print(confusion_matrix(sub_y_test, p))
    print()
    print(f1_score(sub_y_test,p))
    print()
    print(roc_auc_score(sub_y_test, clf.predict_proba(sub_x_test)[:,1]))
    print("-------------------------------------")

i:  4

[[11408    79]
 [  281    43]]

0.19282511210762332

0.7248549621848424
-------------------------------------
i:  5

[[11304   183]
 [  225    99]]

0.32673267326732675

0.7444792664171092
-------------------------------------
i:  6

[[11452    35]
 [  264    60]]

0.2863961813842482

0.7398044971932844
-------------------------------------
i:  7

[[11402    85]
 [  268    56]]

0.24086021505376345

0.7537844713347457
-------------------------------------
i:  8

[[11433    54]
 [  269    55]]

0.2540415704387991

0.8075082460365824
-------------------------------------
i:  9

[[11438    49]
 [  233    91]]

0.3922413793103448

0.7765217954381067
-------------------------------------
i:  10

[[11358   129]
 [  245    79]]

0.29699248120300753

0.7931310165973988
-------------------------------------
i:  11

[[11386   101]
 [  239    85]]

0.3333333333333333

0.7737505467802035
-------------------------------------
i:  12

[[11452    35]
 [  263    61]]

0.29047619047619044

0.821

In [ ]:
forest.score(X_, y)

NotFittedError: This RandomForestClassifier instance is not fitted yet. Call 'fit' with appropriate arguments before using this estimator.

In [ ]:
from sklearn.model_selection import cross_val_score

scores = cross_val_score(forest, X_, y, cv=5)

np.mean(scores)


0.9701052385780764

In [ ]:
ailsatest = pd.read_csv('Data/ailsa/Ailsatest2.csv')

In [ ]:
X_test = first_pipeline.transform(ailsatest)
X_test = final_pipeline.transform(X_test)

y_preds = forest.predict(X_test)

In [ ]:
np.sum(y_preds)

7341

In [ ]:
y_preds_df = pd.DataFrame({'TransactionID': list(ailsatest['TransactionID']), 'isFraud': y_preds})

y_preds_df.head()

,TransactionID,isFraud
0,3663549,0
1,3663550,0
2,3663551,0
3,3663552,0
4,3663553,0


In [ ]:
y_preds_df.to_csv('Data/ailsa/finaloutput.csv', index=False)

In [ ]:
np.sum(y_preds) / len(y_preds)

0.014488119978448403

In [ ]:
np.sum(y) / len(y)

0.03499012764631573